# GOterm Annotation

In this notebook, I'll annotate the CpG background and DML lists from `methylKit` and `DSS` with GOterms. I will use these annotations for gene enrichment.

1. Extract GOterm information from GFF file
2. Create master annotation table
2. Match CpG background and DML lists with GOterms
3. Modify lists for downstream gene enrichment

## 0. Prepare to run script

### 0a. Set working directory

In [45]:
pwd

'/Users/yaamini/Documents/project-oyster-oa/code/Haws'

In [46]:
cd ../../analyses/

/Users/yaamini/Documents/project-oyster-oa/analyses


In [3]:
!mkdir Haws_08-GOterm-annotation

In [47]:
cd Haws_08-GOterm-annotation/

/Users/yaamini/Documents/project-oyster-oa/analyses/Haws_08-GOterm-annotation


## 0b. Install [`GFFutils`](https://gffutils.readthedocs.io/en/v0.12.0/index.html)

This is the program I will use to parse the GFF before retrieving GOterm information. I downloaded the latest release from [this link](https://github.com/fls-bioinformatics-core/GFFUtils/releases).

In [50]:
#Uncompress GFFutils
!tar xzf /Users/Shared/Apps/GFFUtils-0.12.0.tar.gz

In [52]:
#Check requirements
!pip install -r /Users/Shared/Apps/GFFUtils-0.12.0/requirements.txt

  Cloning https://github.com/fls-bioinformatics-core/genomics.git to /private/var/folders/m7/jdh7hbrn2c90384jzrhjbv0h0000gz/T/pip-build-_n7tklft/genomics-bcftbx
    100% |████████████████████████████████| 61kB 4.9MB/s 
    100% |████████████████████████████████| 829kB 791kB/s 
  Running setup.py bdist_wheel for future ... - \ | / - done
  Stored in directory: /Users/yaamini/Library/Caches/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Running setup.py install for genomics-bcftbx ... - \ | / - done
You are using pip version 8.1.2, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [54]:
#Install GFFUtils
!pip install /Users/Shared/Apps/GFFUtils-0.12.0/

Processing /Users/Shared/Apps/GFFUtils-0.12.0
  Running setup.py bdist_wheel for GFFUtils ... - \ done
  Stored in directory: /Users/yaamini/Library/Caches/pip/wheels/14/16/39/8014245fb83cc22925e3c96a965c24dde959b285e961f46c26
Successfully built GFFUtils
You are using pip version 8.1.2, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [55]:
#Upgrade pip based on warnings
!pip install --upgrade pip

    100% |████████████████████████████████| 1.6MB 434kB/s 
  Found existing installation: pip 8.1.2
    Uninstalling pip-8.1.2:
      Successfully uninstalled pip-8.1.2


In [56]:
!ls /Users/Shared/Apps/GFFUtils-0.12.0/

CHANGELOG.rst    README.rst       extras           test
GFFUtils         docs             requirements.txt
LICENSE          examples         setup.py


## 0c. Set variables

In [60]:
# Set directories, input/output files
%env data_dir=/Users/yaamini/Documents/project-gigas-oa-meth/genome-feature-files/ncbi-genomes-2021-05-04

%env analysis_dir=/Users/yaamini/Documents/project-oyster-oa/analyses/Haws_08-GOterm-annotation
analysis_dir="/Users/yaamini/Documents/project-oyster-oa/analyses/Haws_08-GOterm-annotation"

# Input GFF
%env orig_gff=GCF_902806645.1_cgigas_uk_roslin_v1_genomic.gff

# UniProt batch output
%env perl_output=20210602_cgigas_roslin_uniprot_batch_results.txt

# GTF extractor output
%env gtf_extractor_output=20210602_cgigas_roslin_chrom-start-end-Dbxref.csv

# Gene name list for UniProt batch submission
%env gene_list=20210602_cgigas_roslin_gene-list.txt

# Parsed UniProt
%env parsed_uniprot=20210602_cgigas_roslin_accession-gene_id-gene-gene_description-go_ids.csv

# Final output
%env joined_output=20210602_cgigas_roslin_chrom-gene_id_start-end-acc-gene-gene_description-go_ids.csv

env: data_dir=/Users/yaamini/Documents/project-gigas-oa-meth/genome-feature-files/ncbi-genomes-2021-05-04
env: analysis_dir=/Users/yaamini/Documents/project-oyster-oa/analyses/Haws_08-GOterm-annotation
env: orig_gff=GCF_902806645.1_cgigas_uk_roslin_v1_genomic.gff
env: perl_output=20210602_cgigas_roslin_uniprot_batch_results.txt
env: gtf_extractor_output=20210602_cgigas_roslin_chrom-start-end-Dbxref.csv
env: gene_list=20210602_cgigas_roslin_gene-list.txt
env: parsed_uniprot=20210602_cgigas_roslin_accession-gene_id-gene-gene_description-go_ids.csv
env: joined_output=20210602_cgigas_roslin_chrom-gene_id_start-end-acc-gene-gene_description-go_ids.csv


## 1. Extract GOterm information

This is modified from a [script Sam has used previously](https://nbviewer.jupyter.org/github/RobertsLab/code/blob/master/notebooks/sam/20210601_ssal_gff-annotations.ipynb).

### 1a. Use GFFutils to extract gene features

In [61]:
%%bash
head -n 20 "${data_dir}"/"${orig_gff}"

##gff-version 3
#!gff-spec-version 1.21
#!processor NCBI annotwriter
#!genome-build cgigas_uk_roslin_v1
#!genome-build-accession NCBI_Assembly:GCF_902806645.1
#!annotation-source NCBI Crassostrea gigas Annotation Release 102
##sequence-region NC_047559.1 1 55785328
##species https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=29159
NC_047559.1	RefSeq	region	1	55785328	.	+	.	ID=NC_047559.1:1..55785328;Dbxref=taxon:29159;Name=1;gbkey=Src;genome=chromosome;linkage-group=1;mol_type=genomic DNA
NC_047559.1	Gnomon	gene	9839	11386	.	+	.	ID=gene-LOC117693020;Dbxref=GeneID:117693020;Name=LOC117693020;gbkey=Gene;gene=LOC117693020;gene_biotype=lncRNA
NC_047559.1	Gnomon	lnc_RNA	9839	11386	.	+	.	ID=rna-XR_004604272.1;Parent=gene-LOC117693020;Dbxref=GeneID:117693020,Genbank:XR_004604272.1;Name=XR_004604272.1;gbkey=ncRNA;gene=LOC117693020;model_evidence=Supporting evidence includes similarity to: 1 EST%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 1

In [62]:
%%bash
# Extract just gene features
# Extract chromosome name, start, end, and Dbxref fields
# Dbxref is the NCBI gene name, in this particular instance
# Specify input as GFF
# Use awk to format as comma-delimited output to help with downstream parsing/joining
time \
gtf_extract \
--feature gene \
--fields=chrom,start,end,Dbxref \
--gff ${data_dir}/${orig_gff} \
| awk 'BEGIN { OFS = ","; FS="[\t:]"} {print $1, $2, $3, $5}' \
> ${analysis_dir}/${gtf_extractor_output}


real	3m45.478s
user	3m44.736s
sys	0m0.688s


In [63]:
%%bash
#Check output: columns are chr, start, end, Dbrexf (gene ID)
cd "${analysis_dir}"
ls -ltrh ${gtf_extractor_output}

echo ""

head ${gtf_extractor_output}

-rw-r--r--  1 yaamini  staff   1.4M Jun  2 09:49 20210602_cgigas_roslin_chrom-start-end-Dbxref.csv

NC_047559.1,9839,11386,117693020
NC_047559.1,14114,15804,109621113
NC_047559.1,16867,19160,117687066
NC_047559.1,60036,74623,117689737
NC_047559.1,80321,86124,117687025
NC_047559.1,97820,126387,105333279
NC_047559.1,139293,141527,105321471
NC_047559.1,142829,146045,105321472
NC_047559.1,151758,185673,117687070
NC_047559.1,167270,168430,117689460


In [65]:
%%bash
#Confirm GFFutils output is okay by counting gene features in original and extracted files

# Count gene features via GFFutils
echo "GFFutils number of extracted genes:"
gtf_extract -f gene --fields=Dbxref --gff ${data_dir}/${orig_gff} | wc -l

echo ""

# Count gene features via awk
echo "awk number of extracted genes:"
awk '$3 == "gene" { print $0 }' ${data_dir}/${orig_gff} | wc -l

GFFutils number of extracted genes:
   38208

awk number of extracted genes:
   38208


### 1b. Extract Gene IDs for batch submission to UniProt

In [66]:
%%bash

#Take 4th column (gene IDs) and save as a new list

cd "${analysis_dir}"
awk -F"," '{print $4}' "${gtf_extractor_output}" > "${gene_list}"

In [67]:
%%bash

#Check gene list

cd "${analysis_dir}"
head "${gene_list}"

117693020
109621113
117687066
117689737
117687025
105333279
105321471
105321472
117687070
117689460


### 1c. Batch submission to UniProt

I created this script based on one used in Sam's code.

### 1a. Format `DIAMOND blastx` output

In [11]:
#Download blastx output
!wget https://gannet.fish.washington.edu/spartina/project-oyster-oa/data/Cg-roslin/20210601-cgigas-roslin-mito-blastx.outfmt6 \
--no-check-certificate

--2021-06-02 00:33:20--  https://gannet.fish.washington.edu/spartina/project-oyster-oa/data/Cg-roslin/20210601-cgigas-roslin-mito-blastx.outfmt6
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 18030 (18K)
Saving to: ‘20210601-cgigas-roslin-mito-blastx.outfmt6’

20210601-cgigas-ros 100%[===================>]  17.61K  --.-KB/s    in 0s      

2021-06-02 00:33:20 (58.5 MB/s) - ‘20210601-cgigas-roslin-mito-blastx.outfmt6’ saved [18030/18030]



In [13]:
#Check output
!head 20210601-cgigas-roslin-mito-blastx.outfmt6
!wc -l 20210601-cgigas-roslin-mito-blastx.outfmt6

NC_047559.1	sp|B3EWZ6|MLRP2_ACRMI	37.9	2994	1602	80	21150970	21159747	2198	5003	0.0e+00	1859.7
NC_047560.1	sp|O00487|PSDE_HUMAN	92.0	311	24	1	50720737	50721669	1	310	1.9e-154	562.8
NC_047561.1	sp|Q9BLC5|HSP83_BOMMO	63.0	884	122	8	16160352	16157707	2	682	4.2e-269	943.3
NC_047562.1	sp|B0R0T1|VWA8_DANRE	52.1	1531	641	23	38778737	38783224	100	1572	0.0e+00	1490.7
NC_047563.1	sp|Q9XTL9|PYG_DROME	58.1	943	282	8	36595508	36592686	10	841	7.3e-303	1055.8
NC_047564.1	sp|Q8TE73|DYH5_HUMAN	34.5	3018	734	59	38581967	38590942	609	2410	0.0e+00	1333.5
NC_047565.1	sp|Q9GV77|FREM2_LYTVA	47.1	1655	847	12	11073993	11069068	43	1682	0.0e+00	1505.3
NC_047566.1	sp|P41827|HSP74_ANOAL	75.8	637	145	2	23460821	23462704	4	640	9.4e-277	968.8
NC_047567.1	sp|P68362|TBA1A_CRIGR	98.7	376	5	0	28131302	28132429	76	451	1.4e-214	761.5
NC_047568.1	sp|P23098|DYHC_TRIGR	34.4	6578	2029	164	12890443	12910005	117	4465	0.0e+00	2749.5
     209 20210601-cgigas-roslin-mito-blastx.outfmt6


In [14]:
#convert pipes to tab
!tr '|' '\t' < 20210601-cgigas-roslin-mito-blastx.outfmt6 \
> cgigas-roslin-mito-blastx.outfmt6.codeIsolated

In [15]:
!head cgigas-roslin-mito-blastx.outfmt6.codeIsolated

NC_047559.1	sp	B3EWZ6	MLRP2_ACRMI	37.9	2994	1602	80	21150970	21159747	2198	5003	0.0e+00	1859.7
NC_047560.1	sp	O00487	PSDE_HUMAN	92.0	311	24	1	50720737	50721669	1	310	1.9e-154	562.8
NC_047561.1	sp	Q9BLC5	HSP83_BOMMO	63.0	884	122	8	16160352	16157707	2	682	4.2e-269	943.3
NC_047562.1	sp	B0R0T1	VWA8_DANRE	52.1	1531	641	23	38778737	38783224	100	1572	0.0e+00	1490.7
NC_047563.1	sp	Q9XTL9	PYG_DROME	58.1	943	282	8	36595508	36592686	10	841	7.3e-303	1055.8
NC_047564.1	sp	Q8TE73	DYH5_HUMAN	34.5	3018	734	59	38581967	38590942	609	2410	0.0e+00	1333.5
NC_047565.1	sp	Q9GV77	FREM2_LYTVA	47.1	1655	847	12	11073993	11069068	43	1682	0.0e+00	1505.3
NC_047566.1	sp	P41827	HSP74_ANOAL	75.8	637	145	2	23460821	23462704	4	640	9.4e-277	968.8
NC_047567.1	sp	P68362	TBA1A_CRIGR	98.7	376	5	0	28131302	28132429	76	451	1.4e-214	761.5
NC_047568.1	sp	P23098	DYHC_TRIGR	34.4	6578	2029	164	12890443	12910005	117	4465	0.0e+00	2749.5


In [18]:
#Reduce the number of columns using awk: accession code, gene iD, and e-value
#Sort, and save as a new file.
!awk -v OFS='\t' '{print $3, $1, $13}' < cgigas-roslin-mito-blastx.outfmt6.codeIsolated | sort \
> gigas-blast-sort.tab

In [19]:
!head gigas-blast-sort.tab

A0A0R4I9Y1	NW_022994773.1	5.0e-31
A1A5Y0	NW_022994959.1	1.1e-10
A2RRV9	NW_022994962.1	4.8e-37
A5PLL7	NW_022994902.1	2.4e-38
B0R0T1	NC_047562.1	0.0e+00
B3EWZ3	NW_022994974.1	2.6e-09
B3EWZ6	NC_047559.1	0.0e+00
B3MS75	NW_022994960.1	5.6e-25
C3YWU0	NW_022994929.1	1.1e-33
F1R2X6	NW_022994965.1	7.3e-50


### 1b. Join with GOterms

In [ ]:
#Download Uniprot database with GOterm information
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted

In [21]:
!head uniprot-SP-GO.sorted
!wc -l uniprot-SP-GO.sorted

A0A023GPI8	LECA_CANBL	reviewed	Lectin alpha chain (CboL) [Cleaved into: Lectin beta chain; Lectin gamma chain]		Canavalia boliviana	237			mannose binding [GO:0005537]; metal ion binding [GO:0046872]	mannose binding [GO:0005537]; metal ion binding [GO:0046872]	GO:0005537; GO:0046872
A0A023GPJ0	CDII_ENTCC	reviewed	Immunity protein CdiI	cdiI ECL_04450.1	Enterobacter cloacae subsp. cloacae (strain ATCC 13047 / DSM 30054 / NBRC 13535 / NCDC 279-56)	145					
A0A023PXA5	YA19A_YEAST	reviewed	Putative uncharacterized protein YAL019W-A	YAL019W-A	Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)	189					
A0A023PXB0	YA019_YEAST	reviewed	Putative uncharacterized protein YAR019W-A	YAR019W-A	Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)	110					
A0A023PXB5	IRC2_YEAST	reviewed	Putative uncharacterized membrane protein IRC2 (Increased recombination centers protein 2)	IRC2 YDR112W	Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)	102		integ

In [22]:
#Join the first column in the first file with the first column in the second file
#The files are tab delimited, and the output should also be tab delimited (-t $'\t')
!join -1 1 -2 1 -t $'\t' \
gigas-blast-sort.tab \
uniprot-SP-GO.sorted \
> gigas-blast-annot.tab
!head gigas-blast-annot.tab
!wc -l gigas-blast-annot.tab

A0A0R4I9Y1	NW_022994773.1	5.0e-31	R213B_DANRE	reviewed	E3 ubiquitin-protein ligase rnf213-beta (EC 2.3.2.27) (EC 3.6.4.-) (Mysterin-B) (Mysterin-beta) (RING finger protein 213-B) (RING finger protein 213-beta) (RING-type E3 ubiquitin transferase rnf213-beta)	rnf213b	Danio rerio (Zebrafish) (Brachydanio rerio)	5061		cytosol [GO:0005829]	ATPase activity [GO:0016887]; metal ion binding [GO:0046872]; ubiquitin-protein transferase activity [GO:0004842]	cytosol [GO:0005829]; ATPase activity [GO:0016887]; metal ion binding [GO:0046872]; ubiquitin-protein transferase activity [GO:0004842]	GO:0004842; GO:0005829; GO:0016887; GO:0046872
A1A5Y0	NW_022994959.1	1.1e-10	NELL2_DANRE	reviewed	Protein kinase C-binding protein NELL2 (NEL-like protein 2)	nell2 zgc:158375	Danio rerio (Zebrafish) (Brachydanio rerio)	811		extracellular region [GO:0005576]	calcium ion binding [GO:0005509]	extracellular region [GO:0005576]; calcium ion binding [GO:0005509]	GO:0005509; GO:0005576
A2RRV9	NW_022994962.1	4.8e-37	

In [26]:
#Extract columns 1 (accession), 2 (gene ID), and 14 (GOterms)
#Save output
!cut -f1,2,14 gigas-blast-annot.tab \
> _blast-annot.tab
!head _blast-annot.tab

A0A0R4I9Y1	NW_022994773.1	GO:0004842; GO:0005829; GO:0016887; GO:0046872
A1A5Y0	NW_022994959.1	GO:0005509; GO:0005576
A2RRV9	NW_022994962.1	GO:0003954; GO:0016226; GO:0046872; GO:0051536; GO:0051539; GO:0097361
A5PLL7	NW_022994902.1	GO:0005737; GO:0005783; GO:0005789; GO:0016021; GO:0031625; GO:0061630
B0R0T1	NC_047562.1	GO:0005524; GO:0016887
B3EWZ3	NW_022994974.1	GO:0016021
B3EWZ6	NC_047559.1	GO:0005576; GO:0016020
B3MS75	NW_022994960.1	GO:0000339; GO:0005846; GO:0006370; GO:0030422; GO:0031053; GO:0045071; GO:0045292; GO:0051028; GO:0071011; GO:0071013
C3YWU0	NW_022994929.1	GO:0004560; GO:0005576; GO:0006004
F1R2X6	NW_022994965.1	GO:0005634; GO:0005829; GO:0005839; GO:0006281; GO:0006974; GO:0010499; GO:0016504; GO:0016607; GO:0035093; GO:0070577; GO:0070628; GO:1990111


In [29]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having

# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [30]:
!head _blast-GO-unfolded.tab

A0A0R4I9Y1	NW_022994773.1	GO:0004842
A0A0R4I9Y1	NW_022994773.1	GO:0005829
A0A0R4I9Y1	NW_022994773.1	GO:0016887
A0A0R4I9Y1	NW_022994773.1	GO:0046872
A1A5Y0	NW_022994959.1	GO:0005509
A1A5Y0	NW_022994959.1	GO:0005576
A2RRV9	NW_022994962.1	GO:0003954
A2RRV9	NW_022994962.1	GO:0016226
A2RRV9	NW_022994962.1	GO:0046872
A2RRV9	NW_022994962.1	GO:0051536


In [31]:
#Reorganize and sort columns
!awk '{print $3"\t"$2}' _blast-GO-unfolded.tab | gsort -V > _blast-GO-unfolded.sorted

In [32]:
!head _blast-GO-unfolded.sorted

GO:0000002	NW_022994823.1
GO:0000002	NW_022994855.1
GO:0000002	NW_022994858.1
GO:0000002	NW_022994869.1
GO:0000002	NW_022994889.1
GO:0000049	NW_022994958.1
GO:0000064	NW_022994936.1
GO:0000066	NW_022994936.1
GO:0000122	NW_022994808.1
GO:0000122	NW_022994907.1


### 1c. Join with GO Slim terms

In [27]:
#Get GO to GOSlim matching
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0   102k      0  0:00:22  0:00:22 --:--:--  290k


In [28]:
!head GO-GOslim.sorted
!wc -l GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000003	reproduction	other biological processes	P
GO:0000006	high affinity zinc uptake transmembrane transporter activity	transporter activity	F
GO:0000007	low-affinity zinc ion transmembrane transporter activity	transporter activity	F
GO:0000009	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	single strand break repair	DNA metabolism	P
GO:0000012	single strand break repair	stress response	P
   30796 GO-GOslim.sorted


In [37]:
#Join files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $2, $1, $4, $5}' \
> Blastquery-GOslim.tab
!head Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

NW_022994823.1	GO:0000002	cell organization and biogenesis	P
NW_022994855.1	GO:0000002	cell organization and biogenesis	P
NW_022994858.1	GO:0000002	cell organization and biogenesis	P
NW_022994869.1	GO:0000002	cell organization and biogenesis	P
NW_022994889.1	GO:0000002	cell organization and biogenesis	P
NW_022994958.1	GO:0000049	nucleic acid binding activity	F
NW_022994936.1	GO:0000064	transporter activity	F
NW_022994936.1	GO:0000066	transport	P
NW_022994808.1	GO:0000122	RNA metabolism	P
NW_022994907.1	GO:0000122	RNA metabolism	P
    1769 Blastquery-GOslim.tab


## 2. Match CpG background and DML lists with GOterms